# RAVDESS Dataset Starter Notebook

This notebook demonstrates how to load, explore, and process audio data from the [RAVDESS dataset](https://zenodo.org/record/1188976) for emotion recognition tasks.

---

## 1. Import Required Libraries
Import Python libraries for data handling, audio processing, and visualization.

In [3]:
!pip install librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/260.7 kB ? eta -:--:--  Downloading librosa-0.11.0-py3-none-any.whl (260 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 4.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 4.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.1 MB ? eta -:--:--Collecting soundfile>=0.12.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/164.8 kB ? eta -:--:--Collecting soxr>=0.3.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 6.9 MB/s eta 0:00:00
 

In [4]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt

## 2. Download and Extract RAVDESS Dataset

If you have not already downloaded the dataset, you can get it from [Zenodo](https://zenodo.org/record/1188976). Place the extracted files in the `data/` directory.

## 3. Explore Dataset Structure

List the folders and files in the dataset to understand its organization.

In [5]:
data_dir = 'data/Audio_Song_Actors_01-24'
actors = sorted([d for d in os.listdir(data_dir) if d.startswith('Actor_')])
print('Actors:', actors)
print('Sample files from Actor_01:')
print(os.listdir(os.path.join(data_dir, 'Actor_01'))[:5])

Actors: ['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17']
Sample files from Actor_01:
['03-02-03-02-02-02-01.wav', '03-02-03-01-01-02-01.wav', '03-02-02-02-01-01-01.wav', '03-02-02-01-02-01-01.wav', '03-02-01-01-02-02-01.wav']


## 4. Load Example Audio File

Load a sample `.wav` file from Actor_01 and display its duration and sample rate.

In [6]:
example_file = os.path.join(data_dir, 'Actor_01', os.listdir(os.path.join(data_dir, 'Actor_01'))[0])
y, sr = librosa.load(example_file, sr=None)
print(f'Loaded {example_file}')
print(f'Sample rate: {sr}, Duration: {len(y)/sr:.2f} seconds')

KeyboardInterrupt: 

## 5. Visualize Audio Waveform and Spectrogram

Plot the waveform and Mel-spectrogram of the loaded audio file.

In [ ]:
plt.figure(figsize=(12, 4))
librosa.display.waveshow(y, sr=sr)
plt.title('Waveform')
plt.show()

S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
S_dB = librosa.power_to_db(S, ref=np.max)
plt.figure(figsize=(12, 4))
librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
plt.title('Mel-Spectrogram')
plt.colorbar(format='%+2.0f dB')
plt.show()

## 6. Extract Audio Features (MFCCs, Mel-spectrograms)

Extract MFCCs and Mel-spectrogram features for use in machine learning models.

In [ ]:
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
print('MFCCs shape:', mfccs.shape)
plt.figure(figsize=(10, 4))
librosa.display.specshow(mfccs, x_axis='time')
plt.colorbar()
plt.title('MFCC')
plt.tight_layout()
plt.show()

## 7. Parse Filenames for Emotion Labels

Extract emotion labels from filenames using the RAVDESS naming convention.

In [ ]:
def parse_ravdess_filename(filename):
    # Example: '03-01-01-01-01-01-01.wav'
    # 3rd group is emotion: 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
    emotion_map = {
        '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
        '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
    }
    parts = filename.split('-')
    emotion_code = parts[2]
    return emotion_map.get(emotion_code, 'unknown')

sample_filename = os.listdir(os.path.join(data_dir, 'Actor_01'))[0]
print(f'Filename: {sample_filename}, Emotion: {parse_ravdess_filename(sample_filename)}')